# Explore Dynamic Dataset

Make lists of keys needed:

In [1]:
datasets = ['23504-5-4']

keys = []
for d in datasets:
    animal, session, scan = d.split('-')
    keys.append({
     'animal_id': animal,
     'session': session,
     'scan_idx': scan,
     'preproc_id': 0
    })

Here I demonstrate how to fill tables necessary for dynamic dataset generation. 

In [2]:
from neuro_data.movies import data_schemas, configs, stats

from neuro_data.movies.data_schemas import *
meso = dj.create_virtual_module('meso', 'pipeline_meso')
import datajoint as dj


import numpy as np

Connecting synicix@at-database1.ad.bcm.edu:3306


First thing first, make sure to insert your desired sessions into `MovieScanCandidate` table. Best would be to fill the `keys` list of dict like above and use the following line to insert.

In [3]:
data_schemas.MovieScanCandidate().insert(fuse.ScanDone & keys, ignore_extra_fields=True, skip_duplicates=True)

Verify that your desired candidates are actually inserted

Now simply execute the tables below **in the following order**

In [4]:
data_schemas.MovieScan.populate(keys, display_progress=True)

100%|██████████| 1/1 [00:00<00:00, 11.50it/s]


In [5]:
data_schemas.ConditionTier.populate(keys, display_progress=True)

  0%|          | 0/1 [00:00<?, ?it/s]

21-02-2020:08:20:43 INFO     data_schemas.py      200:	 Processing {'animal_id': 23504, 'session': 5, 'scan_idx': 4, 'pipe_version': 1, 'segmentation_method': 6, 'spike_method': 5}


100%|██████████| 1/1 [00:00<00:00, 26.45it/s]


In [6]:
data_schemas.MovieClips.populate(keys, display_progress=True)

0it [00:00, ?it/s]


In [7]:
data_schemas.InputResponse.populate(keys, display_progress=True)

0it [00:00, ?it/s]


In [8]:
data_schemas.Eye.populate(keys)

In [9]:
data_schemas.Eye2.populate(keys)

In [10]:
data_schemas.Treadmill.populate(keys)

In [11]:
anatomy.AreaMembership() & keys

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,pipe_version,segmentation_method,unit_id,brain_area short name for cortical area
20457,1,15,1,6,1,V1
20457,1,15,1,6,2,V1
20457,1,15,1,6,3,V1
20457,1,15,1,6,4,V1
20457,1,15,1,6,5,V1
20457,1,15,1,6,6,V1
20457,1,15,1,6,7,V1
20457,1,15,1,6,8,V1
20457,1,15,1,6,9,V1
20457,1,15,1,6,10,V1


In [12]:
anatomy.LayerMembership() & keys

animal_id id number,session session index for the mouse,scan_idx number of TIFF stack file,pipe_version,segmentation_method,unit_id unique per scan & segmentation method,layer short name for cortical area
20457,1,15,1,6,1,L2/3
20457,1,15,1,6,2,L2/3
20457,1,15,1,6,3,L2/3
20457,1,15,1,6,4,L2/3
20457,1,15,1,6,5,L2/3
20457,1,15,1,6,6,L2/3
20457,1,15,1,6,7,L2/3
20457,1,15,1,6,8,L2/3
20457,1,15,1,6,9,L2/3
20457,1,15,1,6,10,L2/3


If you know that they are from same area/layer, then you can use the following to manually fill the area and layer memberhsip table

In [13]:
units = (meso.ScanSet.Unit & keys).fetch('KEY')

for u in units:
    u['brain_area'] = 'V1'
    u['layer'] = 'L2/3'

anatomy.AreaMembership.insert(units, ignore_extra_fields=True, allow_direct_insert=True, skip_duplicates=True)

anatomy.LayerMembership.insert(units, ignore_extra_fields=True, allow_direct_insert=True, skip_duplicates=True)

## To generate new entry into MovieMultiDataset

Run the following specifying the key, one at a time. **CAUTION**: it can also create a group with more than one key in it at a time, so carefully read the prompt! Please also be sure to give a meaningful description of the data group.

Register each key entry separtely

In [14]:
for key in keys:
    name = '{animal_id}-{session}-{scan_idx}'.format(**key)
    MovieMultiDataset().add_entry(name, key)

Already found entry with group_id=20 correspondig to the specified members:
{'animal_id': 20457, 'session': 1, 'scan_idx': 15, 'pipe_version': 1, 'segmentation_method': 6, 'spike_method': 5, 'preproc_id': 0}
Aborting...


## Generate HDF5 files

To trigger the generation of HDF5 files, run the following, but only upon ensuring that you have all above tables filled!

In [17]:
for key in keys:
    key['preproc_id'] = 0
    #print('Generating HDF5 file for {}'.format(key))
    filename = data_schemas.InputResponse().get_filename(key)
    #print('Data for {} can be found in {}'.format(key, filename))
    print(filename)

/external/cache/movies20457-1-15-pre0-pipe1-seg6-spike5.h5


## Fill out the Oracle score

In [ ]:
data_hash = dict(data_hash = "ecb7c24fafd19503a2eef756ac4a24a4")

In [ ]:
configs.DataConfig.AreaLayer & data_hash

In [ ]:
stats.Oracle.populate(MovieMultiDataset.Member & keys, data_hash)